Depositos para enviar, orderm de preferencia:
musical_matriz
musical_filal
em seguida onde houver mais estoque


onde days_available = 0 criar observação onde informa que há muito tempo sem estoque

In [1]:
import json
import os
import time
from pandas import json_normalize
import psycopg2
from psycopg2 import sql
import math
import pandas as pd
import requests
from dotenv import load_dotenv
from datetime import datetime, timedelta
import numpy as np

load_dotenv()

ACCESS_TOKEN_MCENTER = os.getenv("ACCESS_TOKEN_MCENTER")
ACCESS_TOKEN_BUENOSHOPS = os.getenv("ACCESS_TOKEN_BUENOSHOPS")
ACCESS_TOKEN_MUSICALCRIS = os.getenv("ACCESS_TOKEN_MUSICALCRIS")

HOST = os.getenv("HOST")
POSTGRES_DB = os.getenv("POSTGRES_DB")
POSTGRES_USER = os.getenv("POSTGRES_USER")
POSTGRES_PASSWORD = os.getenv("POSTGRES_PASSWORD")


# Informações de conexão com o banco de dados PostgreSQL
db_config = {
    "host": HOST,
    "database": POSTGRES_DB,
    "user": POSTGRES_USER,
    "password": POSTGRES_PASSWORD,
}

In [2]:
def condf(df, coluna, valor):
    """
    Consulta um DataFrame com base em uma coluna e valor específicos.

    Parâmetros:
    - df: DataFrame a ser consultado.
    - coluna: Nome da coluna para a condição de consulta.
    - valor: Valor desejado na coluna.

    Retorna:
    Um DataFrame contendo apenas as linhas que atendem à condição.
    """
    resultado = df[df[coluna] == valor]
    return resultado


def condf_date(df, coluna_data, data_pesquisada):
    """
    Consulta um DataFrame com base em uma coluna de datas.

    Parâmetros:
    - df: DataFrame a ser consultado.
    - coluna_data: Nome da coluna de datas.
    - data_pesquisada: Data desejada para a consulta.

    Retorna:
    Um DataFrame contendo apenas as linhas que correspondem à data pesquisada.
    """
    resultado = df[pd.to_datetime(df[coluna_data]).dt.date == data_pesquisada]
    return resultado

### Período a consultar

In [3]:
# Defina as datas de início e fim desejadas
data_inicio = datetime(2023, 11, 1).date()
data_fim = datetime(2023, 12, 13).date()
# data_fim = data_fim + timedelta(days=1)  # + 1 dia para pegar a data atual no DB
# data_fim = data_fim - timedelta(days=1)  # + 1 dia para pegar a data atual no DB
print("data_inicio: ", data_inicio)
print("data_fim: ", data_fim)

data_inicio:  2023-11-01
data_fim:  2023-12-13


### Historico de estoque fulfillment

In [4]:
# Buscando histórico de estoque na tabela
try:
    conn = psycopg2.connect(**db_config)

    sql_query = f"SELECT * FROM mcenter_fulfillment_stock WHERE created_at BETWEEN '{data_inicio}' AND '{data_fim};'"
    # sql_query = f"SELECT * FROM fulfillment_stock WHERE created_at BETWEEN '2023-12-04' AND '2023-12-05';"
    print(sql_query)
    df_stock = pd.read_sql(sql_query, conn)

except psycopg2.Error as e:
    print(f"Erro do psycopg2 ao consultar fulfillment_stock: {e}")

except Exception as e:
    print(f"Erro ao consultar fulfillment_stock: {e}")

finally:
    if conn is not None:
        conn.close()

SELECT * FROM mcenter_fulfillment_stock WHERE created_at BETWEEN '2023-11-01' AND '2023-12-13;'


C:\Users\couti\AppData\Local\Temp\ipykernel_5452\1935300295.py:8: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_stock = pd.read_sql(sql_query, conn)


In [5]:
# df_stock
# print(df_stock.shape)
# duplicates = df_stock[df_stock.duplicated(keep=False)]
# duplicates

In [6]:
# # datas consultadas, dias em que um produto pode ou não estar disponível
# # df_stock["created_at"].value_counts().index.to_list()
# df_stock["created_at"].value_counts().sort_index()
# df_stock["created_at"].dt.date.value_counts().sort_index()
# # df_stock["created_at"].dt.date.value_counts()

# # len(df_stock["created_at"].dt.date.value_counts().sort_index())

In [7]:
# df_stock

In [8]:
# # buscando anuncio e verificando dias consultados
# result = df_stock[df_stock['ml_inventory_id'] == 'FBAY80623']
# result = result.sort_values(by='created_at')
# result

In [9]:
# # encontrando anuncios pela data
# df_stock["created_at"] = pd.to_datetime(df_stock["created_at"])
# result = df_stock.loc[df_stock['created_at'].dt.date == pd.to_datetime('2023-12-7').date()]
# result

In [10]:
# df_stock[df_stock['created_at']=='2023-12-07 03:01:03.036386']

In [11]:
# Ordenando stock por data
df_stock = df_stock.sort_values(by="created_at", ascending=False)
df_stock["data"] = df_stock["created_at"].dt.date
df_stock = df_stock.drop(["created_at"], axis=1)

In [12]:
# df_stock

In [13]:
# df_stock.dtypes

In [14]:
# df_stock['data'] = pd.to_datetime(df_stock['data'])

# dfx = condf(df_stock, 'data', '2023-11-1')
# dfx

In [15]:
# # A linha abaixo conta as ocorrências de cada valor em 'ml_inventory_id'
# # value_counts = result['ml_inventory_id'].value_counts()
# value_counts = df_stock['ml_inventory_id'].value_counts()

# # Filtra para incluir apenas aqueles que ocorrem mais de 2 vezes
# filtered_counts = value_counts[value_counts > 1]

# filtered_counts.shape
# # filtered_counts
# value_counts

In [16]:
# df_stock[df_stock["detail_status"] == "transfer"]

In [17]:
# condf(df_stock,'ml_inventory_id','CRMK90073')

In [18]:
# condf(df_stock, "detail_status", "transfer")

In [19]:
## Cria coluna has_stock, se available_quantity <= 0, has_stock= False ##
df_stock = df_stock.assign(has_stock=lambda x: x["available_quantity"] > 0)
df_stock = df_stock.sort_values(by="data", ascending=False).reset_index(drop=True)

In [20]:
# df_stock

In [21]:
# condf(df_stock, "ml_inventory_id", "DSGP06967")

In [22]:
# print(df_stock.shape)
# df_stock = df_stock.drop_duplicates()
# print(df_stock.shape)

#### Dias em que produto esteve disponível

In [23]:
# condf(df_stock,'ml_inventory_id','ABCB20467')

In [24]:
## Contando dias em que produto esteve disponível
days_available = df_stock.groupby(["ml_inventory_id"])["has_stock"].sum().reset_index()
days_available = days_available.rename(columns={"has_stock": "days_available"})

In [25]:
# days_available

In [26]:
# days_available.sort_values(by='days_available',ascending=False)
# days_available['days_available'].value_counts()

# ## Contando dias em que produto esteve disponível
# days_available = df_stock.groupby(["ml_inventory_id","references_variation_id"])["has_stock"].sum().reset_index()
# days_available = days_available.rename(columns={"has_stock": "days_available"})
# days_available

In [27]:
# condf(df_stock, "ml_inventory_id", "MENA66694")

In [28]:
# ## Contando dias em que produto esteve disponível
# days_available_ = df_stock.groupby(["ml_inventory_id","references_variation_id"])["has_stock"].sum().reset_index()
# days_available_ = days_available_.rename(columns={"has_stock": "days_available"})
# days_available_

In [29]:
# condf(days_available, "ml_inventory_id", "DSGP06967")
# condf(days_available, "ml_inventory_id", "DSGP06967")

In [30]:
# Unindo DFs
df_stock_days = df_stock.merge(days_available, on=["ml_inventory_id"], how="inner")
# df_stock_days_left = df_stock.merge(days_available, on=["ml_inventory_id"], how="left")
# df_stock.rename(columns={'days_available_x': 'days_available'}, inplace=True)  # Corrigindo a sintaxe
# df_stock = df_stock.drop(columns='days_available_y', axis=1)  # Corrigindo a sintaxe

# Verificando o formato após a correção
# print(df_stock.shape)
#

In [31]:
# print(df_stock.shape)
# print(df_stock_days.shape)
# print(df_stock_days_left.shape)
# df_stock_days.sample(3)

In [32]:
# # Unindo DFs
# df_stock_ = df_stock.merge(days_available, on=["ml_inventory_id"], how="inner")
# # df_stock_.rename(columns={'days_available_x': 'days_available'}, inplace=True)  # Corrigindo a sintaxe
# # df_stock_ = df_stock_.drop(columns='days_available_y', axis=1)  # Corrigindo a sintaxe

# # Verificando o formato após a correção
# print(df_stock_.shape)
# df_stock_

In [33]:
# data de hoje
data_de_hoje = datetime.now().date()
data_de_hoje = data_de_hoje - timedelta(days=1)
# print(data_de_hoje)

df_stock_days["data"] = pd.to_datetime(df_stock_days["data"])

# Filtra apenas as linhas onde 'data' é igual à data de hoje
df_stock_today = df_stock_days[df_stock_days["data"].dt.date == data_de_hoje]
df_stock_today = df_stock_today.rename(
    columns={"available_quantity": "available_quantity_today"}
)

# Se detail_status = transfer: available_quantity_today = available_quantity_today + detail_quantity
df_stock_today["total_available_quantity"] = df_stock_today.apply(
    lambda row: row["detail_quantity"] + row["available_quantity_today"]
    if row["detail_status"] == "transfer"
    else row["available_quantity_today"],
    axis=1,
)

df_stock_today["total_available_quantity"] = df_stock_today[
    "total_available_quantity"
].astype("int64")

In [34]:
df_stock_today

,ml_inventory_id,available_quantity_today,detail_status,detail_quantity,references_id,references_variation_id,data,has_stock,days_available,total_available_quantity


In [35]:
# Se detail_status = transfer: available_quantity_today = available_quantity_today + detail_quantity
df_stock_today["total_available_quantity"] = df_stock_today.apply(
    lambda row: row["detail_quantity"] + row["available_quantity_today"]
    if row["detail_status"] == "transfer"
    else row["available_quantity_today"],
    axis=1,
)

df_stock_today["total_available_quantity"] = df_stock_today[
    "total_available_quantity"
].astype("int64")

In [36]:
# df_stock_today.sample(5)

In [37]:
# condf(df_stock,'ml_inventory_id','FSNB97805')
# condf(df_stock_today,'ml_inventory_id','FSNB97805')

### Buscando hitorico de orders no BD

In [38]:
# Buscando histórico de vendas na tabela ml_orders_hist para o período definido
try:
    conn = psycopg2.connect(**db_config)

    sql_query = f"SELECT * FROM mcenter_ml_orders WHERE date_closed BETWEEN '{data_inicio}' AND '{data_fim}'"
    print(sql_query)
    df_orders = pd.read_sql(sql_query, conn)

except psycopg2.Error as e:
    print(f"Erro do psycopg2 ao consultar ml_orders_hist: {e}")
    # logger.error(f"Erro do psycopg2 ao consultar ml_orders_hist: {e}")

except Exception as e:
    print(f"Erro ao consultar ml_orders_hist: {e}")
    # logger.error(f"Erro ao consultar ml_orders_hist: {e}")

finally:
    if conn is not None:
        conn.close()

# # Ordenando orders por data
# df_orders = df_orders.sort_values(by="date_approved", ascending=False)
# df_orders["data"] = df_orders["date_approved"].dt.date
# df_orders = df_orders.drop(["date_closed","date_approved"], axis=1)

# filtros
df_orders = df_orders[df_orders["logistic_type"] == "fulfillment"]
# df_orders = df_orders.drop(columns=['category_id','pack_id','variation_attributes_id','variation_name','variation_value_id', 'data'])
df_orders = df_orders.drop(
    columns=[
        "category_id",
        "pack_id",
        "variation_attributes_id",
        "variation_name",
        "variation_value_id",
    ]
)

# change column
df_orders["variation_id"] = df_orders["variation_id"].replace("nan", "0", regex=True)

# df_orders = df_orders[df_orders["order_status"] == "paid"]
# df_orders = df_orders[df_orders["payment_status"] == "approved"]


# df_orders = df_orders.drop(
#     ["pack_id", "date_approved", "order_status", "payment_status"], axis=1
# )
df_orders.rename(columns={"quantity": "sold_quantity"}, inplace=True)

SELECT * FROM mcenter_ml_orders WHERE date_closed BETWEEN '2023-11-01' AND '2023-12-13'


C:\Users\couti\AppData\Local\Temp\ipykernel_5452\2680549919.py:7: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_orders = pd.read_sql(sql_query, conn)


In [39]:
print(df_orders.shape)
df_orders = df_orders.drop_duplicates()
print(df_orders.shape)

(6409, 13)
(5526, 13)


In [40]:
# Ordenando orders por data
df_orders = df_orders.sort_values(by="date_approved", ascending=False)
df_orders["data"] = df_orders["date_approved"].dt.date
df_orders = df_orders.drop(["date_closed", "date_approved"], axis=1)

In [41]:
df_orders.sample(3)

,ml_code,variation_id,seller_sku,order_id,sold_quantity,title,order_status,payment_status,shipping_id,variation_value_name,logistic_type,data
7680,MLB2770292345,175608239370,FULLVANDOREN2SR2115,2000007113324068,1,"2 Palheta Sax Alto Vandoren Tradicional 1 1,5 ...",paid,approved,42893298480,"1,5",fulfillment,2023-12-10
573,MLB3435228689,0,FULLP10LNINJA10FT,2000006820689384,1,Cabo Guitarra Violão Santo Angelo Ninja P10 Em...,paid,approved,42767671082,None,fulfillment,2023-11-05
7892,MLB3434144067,0,FULLP10LNINJA15FT,2000007122496172,1,"Cabo Santo Angelo Ninja L 4,57 Metros Cabos P/...",paid,approved,42897086539,None,fulfillment,2023-12-11


In [42]:
# df_orders.head()

In [43]:
# df_orders
# df_orders['pack_id'].value_counts()

# # condf(df_orders,'pack_id','2000005059931419')

In [44]:
# print(df_orders.shape)
# df_orders.head(3)

In [45]:
# condf(df_orders, "ml_code", "MLB1992541482")

#### Total de vendas por ml_code e variation_id

In [46]:
# df_orders.shape

In [47]:
# df_orders

In [48]:
# condf(df_orders,'ml_code','MLB3516894673')

In [49]:
# Total de vendas por ml_code e id de variação
df_orders_quantity = (
    df_orders.groupby(["ml_code", "variation_id"])["sold_quantity"].sum().reset_index()
)

In [50]:
df_orders_quantity

,ml_code,variation_id,sold_quantity
0,MLB1363602423,0,24
1,MLB1398114866,174301393967,1
2,MLB1398114866,174302125663,2
3,MLB1399101853,0,1
4,MLB1400355425,0,26
...,...,...,...
336,MLB4202595608,0,28
337,MLB4217370456,0,1
338,MLB4237823132,0,6
339,MLB924922735,0,14


In [51]:
print(f"Número de ml_code únicos: {len(df_orders_quantity['ml_code'].unique())}")
print(
    f"Número de variation_id únicos: {len(df_orders_quantity['variation_id'].unique())}"
)

Número de ml_code únicos: 269
Número de variation_id únicos: 131


In [52]:
# condf(df_orders_quantity,'ml_code','MLB2075516288')
# df_orders_quantity['ml_code'].value_counts()

In [53]:
# Acrescentando total de vendas ao DF
df_total_sales = pd.merge(
    # df_orders, resultado, on=["ml_code", "variation_id"], how="inner"
    df_orders,
    df_orders_quantity,
    on=["ml_code", "variation_id"],
    how="inner",
)
df_total_sales = df_total_sales.rename(
    columns={"sold_quantity_y": "total_sold_quantity"}
)
df_total_sales = df_total_sales.drop(
    columns=["sold_quantity_x", "order_status", "payment_status"]
)

In [54]:
print(f"Total de vendas = {df_total_sales.shape[0]}")

Total de vendas = 5526


In [55]:
# df_orders.shape

In [56]:
# df_total_sales.columns

In [57]:
# df_total_sales.sample()

In [58]:
# # # Exibir todas as linhas e colunas
# # pd.set_option('display.max_rows', None)
# # pd.set_option('display.max_columns', None)

# # Voltar ao normal
# pd.reset_option('display.max_rows')
# pd.reset_option('display.max_columns')


# # condf(df_total_sales,'ml_code','MLB3427769549')
# # # x = condf(df_orders,'ml_code','MLB3427769549')

# # # x = x.sort_values(by='data')
# # # x.head(15)
# # x = df_orders[df_orders['ml_code']=='MLB3427769549']
# # x['data'] = pd.to_datetime(x['data'])
# # x[x['data']=='2023-11-22']
# # # x['sold_quantity'].value_counts()

# # # counts = x[x['sold_quantity'] > 1]['sold_quantity'].value_counts()
# # # counts

# # y = x.groupby('data')['sold_quantity'].sum().reset_index()
# # y

# # # # x

In [59]:
# x.dtypes

In [60]:
# df_total_sales['data'].value_counts()

In [61]:
# condf(df_total_sales, "ml_code", "MLB4127982748")
# condf(df_total_sales, "ml_code", "MLB4127982748")

In [62]:
# df_total_sales = df_total_sales.drop(["sold_quantity_x", "shipping_id", "data"], axis=1)
# df_total_sales = df_total_sales.drop_duplicates()

In [63]:
# condf(df_total_sales, "ml_code", "MLB1992541482")

Neste ponto temos o total de itens vendidos de um anúncio por período e a quantidade de dias em que um produto esteve disponível.
precisamos juntar esses dados para calcular, para isso trarei as informações de produtos

In [64]:
# print(df_total_sales.shape)
# df_total_sales.sample()

In [65]:
# condf(df_total_sales, 'ml_code', 'MLB3461718736')
# condf(df_orders, 'ml_code', 'MLB3461718736')
# # condf(resultado, 'ml_code', 'MLB3427769549')
# # condf(df_stock, 'ml_code', 'MLB3427769549')

In [66]:
# print(df_stock_today.shape)
# df_stock_today.sample()

#### Buscando Produtos

In [67]:
# Buscando dados de produtos na tabela tiny_fulfillment
try:
    conn = psycopg2.connect(**db_config)
    sql_query = "SELECT * FROM mcenter_items"
    df_codes = pd.read_sql(sql_query, conn)
except psycopg2.Error as e:
    # logger.error(f"Erro do psycopg2 ao consultar fulfillment_stock: {e}")
    print(f"Erro do psycopg2 ao consultar tiny_fulfillment: {e}")

except Exception as e:
    # logger.error(f"Erro ao consultar tabela tiny_fulfillment: {e}")
    print(f"Erro ao consultar tabela tiny_fulfillment: {e}")

finally:
    if conn is not None:
        conn.close()

# df_codes["ml_code"] = df_codes["ml_code"].apply(lambda x: "MLB" + str(x))
df_codes.rename(columns={"inventory_id": "ml_inventory_id"}, inplace=True)
df_codes = df_codes.drop(["created_at", "updated_at"], axis=1)

C:\Users\couti\AppData\Local\Temp\ipykernel_5452\3854357628.py:5: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_codes = pd.read_sql(sql_query, conn)


In [68]:
df_not_catalogo = df_codes[df_codes["catalog_listing"] == False]
df_catalogo = df_codes[df_codes["catalog_listing"] == True]

In [69]:
df_not_catalogo.sample()

,ml_code,ml_inventory_id,value_name,variation_id,status,catalog_listing,logistic_type
168,MLB1465231671,MOPJ68127,Amarelo,52181329446,active,False,fulfillment


In [70]:
df_catalogo.sample()

,ml_code,ml_inventory_id,value_name,variation_id,status,catalog_listing,logistic_type
166,MLB3778561802,AVOX05934,NaN,NaN,closed,True,fulfillment


In [71]:
df_not_catalogo

,ml_code,ml_inventory_id,value_name,variation_id,status,catalog_listing,logistic_type
0,MLB1435852259,TLCG38735,NaN,NaN,active,False,fulfillment
1,MLB1435836214,PUOF38682,NaN,NaN,active,False,fulfillment
2,MLB1400355425,ZXFT21237,NaN,NaN,active,False,fulfillment
3,MLB1440291959,JGRW38703,NaN,NaN,active,False,fulfillment
4,MLB1564998298,OVRP11252,NaN,NaN,paused,False,fulfillment
...,...,...,...,...,...,...,...
527,MLB1490210934,SPKV63227,nan,NaN,paused,False,fulfillment
528,MLB1556975175,DZVJ05686,Marrom,57895096313,active,False,fulfillment
529,MLB950338822,MAEG54107,nan,NaN,paused,False,fulfillment
530,MLB1662970458,UIUW89800,Preto,64498487033,paused,False,fulfillment


In [72]:
print(df_not_catalogo.shape)
print(df_catalogo.shape)

(361, 7)
(171, 7)


In [73]:
df_catalogo

,ml_code,ml_inventory_id,value_name,variation_id,status,catalog_listing,logistic_type
19,MLB2170026503,ZNFW24350,NaN,NaN,paused,True,fulfillment
26,MLB3435374559,RYWV54124,NaN,NaN,paused,True,fulfillment
27,MLB3435352761,QYGO52123,NaN,NaN,paused,True,fulfillment
28,MLB3435350955,HGVR43626,NaN,NaN,active,True,fulfillment
29,MLB3435245361,PKNN52962,NaN,NaN,paused,True,fulfillment
...,...,...,...,...,...,...,...
519,MLB4127982748,RLJX42857,nan,NaN,paused,True,fulfillment
520,MLB3497394073,XGSW66673,nan,NaN,paused,True,fulfillment
521,MLB4191526348,LXOO59456,nan,NaN,paused,True,fulfillment
522,MLB3521778415,PSMV02965,nan,NaN,paused,True,fulfillment


In [74]:
# df_orders.sample()

In [75]:
df_total_sales_cat = pd.merge(
    df_catalogo,
    df_total_sales,
    left_on=["ml_code"],
    right_on=["ml_code"],
    how="left",
)
df_total_sales_cat = df_total_sales_cat.drop_duplicates()

df_total_sales_not_cat = pd.merge(
    df_not_catalogo,
    df_total_sales,
    left_on=["ml_code", "variation_id"],
    right_on=["ml_code", "variation_id"],
    how="left",
)
df_total_sales_not_cat = df_total_sales_not_cat.drop_duplicates()

In [76]:
df_total_sales_not_cat["total_sold_quantity"] = df_total_sales_not_cat[
    "total_sold_quantity"
].fillna(0)
df_total_sales_cat["total_sold_quantity"] = df_total_sales_cat[
    "total_sold_quantity"
].fillna(0)

df_total_sales_not_cat["total_sold_quantity"] = df_total_sales_not_cat[
    "total_sold_quantity"
].astype("int64")
df_total_sales_cat["total_sold_quantity"] = df_total_sales_cat[
    "total_sold_quantity"
].astype("int64")

In [77]:
# df_total_sales_cat.sample(5)
# df_total_sales_not_cat['total_sold_quantity'].value_counts()
# df_total_sales_cat['total_sold_quantity'].value_counts()

In [78]:
print(df_total_sales_cat.shape)
print(df_total_sales_not_cat.shape)

(3279, 16)
(1899, 15)


In [79]:
# df_total_sales_cat

In [80]:
# df_total_sales_cat = df_total_sales_cat.drop(
#     columns=["data", "shipping_id", "variation_id_x", "order_id"]
# )
df_total_sales_cat_x = df_total_sales_cat.drop(
    columns=["data", "shipping_id", "variation_id_x", "order_id"]
)
df_total_sales_cat_x = df_total_sales_cat_x.drop_duplicates()
df_total_sales_cat_x.shape

(172, 12)

In [81]:
# df_total_sales_cat_x

In [82]:
# df_total_sales_cat['value_name'].value_counts()

In [83]:
# condf(df_total_sales_cat, 'ml_code','MLB3427769549')

In [84]:
# df_vendas_total_sales_not_cat

In [85]:
print(df_total_sales_not_cat.shape)

(1899, 15)


In [86]:
print(df_total_sales_not_cat.shape)
df_total_sales_not_cat_x = df_total_sales_not_cat.drop(
    columns=["data", "shipping_id", "order_id"]
)
df_total_sales_not_cat_x = df_total_sales_not_cat_x.drop_duplicates()
print(df_total_sales_not_cat_x.shape)

(1899, 15)
(362, 12)


In [87]:
# df_total_sales_cat_x

In [88]:
print(df_total_sales_cat_x.shape)
print(df_total_sales_not_cat_x.shape)

(172, 12)
(362, 12)


In [89]:
# df_total_sales_cat['ml_inventory_id'].value_counts()

In [90]:
# Antes de desabilitar
# pd.set_option('display.max_rows', None)
# pd.set_option('display.max_columns', None)
# pd.set_option('display.max_colwidth', None)

# Seu código para exibir o DataFrame

# # Depois de desabilitar
# pd.reset_option('display.max_rows')
# pd.reset_option('display.max_columns')
# pd.reset_option('display.max_colwidth')


# df_total_sales_cat = df_total_sales_cat.drop_duplicates()

df_total_sales_cat_x = df_total_sales_cat.drop_duplicates(
    subset=["ml_code", "ml_inventory_id"]
)
df_total_sales_not_cat_x = df_total_sales_not_cat.drop_duplicates(
    subset=["ml_code", "ml_inventory_id"]
)

In [91]:
# df_orders.shape

In [92]:
# condf(df_total_sales_cat,'ml_inventory_id','WTHI04163')
# condf(y,'ml_inventory_id','WTHI04163')

In [93]:
# condf(df_total_sales_cat,'ml_inventory_id','AWAZ99732')

In [94]:
df_total_cat = df_total_sales_cat_x.copy()
df_total_not_cat = df_total_sales_not_cat_x.copy()

In [95]:
df_total_cat.columns

Index(['ml_code', 'ml_inventory_id', 'value_name', 'variation_id_x', 'status',
       'catalog_listing', 'logistic_type_x', 'variation_id_y', 'seller_sku',
       'order_id', 'title', 'shipping_id', 'variation_value_name',
       'logistic_type_y', 'data', 'total_sold_quantity'],
      dtype='object')

In [96]:
# Somando total de vendas por inventory_id
df_sum_qt_sold_cat = (
    df_total_cat.groupby("ml_inventory_id")["total_sold_quantity"].sum().reset_index()
)
df_sum_qt_sold_cat = df_sum_qt_sold_cat.rename(
    columns={"total_sold_quantity": "total_sold_catalog"}
)

df_total_cat = pd.merge(
    df_total_cat,
    df_sum_qt_sold_cat[["ml_inventory_id", "total_sold_catalog"]],
    on="ml_inventory_id",
    how="left",
)

df_total_cat.rename(columns={"variation_id_y": "variation_id_"})


df_sum_qt_sold_not_cat = (
    df_total_not_cat.groupby("ml_inventory_id")["total_sold_quantity"]
    .sum()
    .reset_index()
)
df_sum_qt_sold_not_cat = df_sum_qt_sold_not_cat.rename(
    columns={"total_sold_quantity": "total_sold_not_catalog"}
)
df_total_not_cat = pd.merge(
    df_total_not_cat,
    df_sum_qt_sold_not_cat[["ml_inventory_id", "total_sold_not_catalog"]],
    on="ml_inventory_id",
    how="left",
)

In [97]:
print(df_total_cat.shape)
print(df_total_not_cat.shape)

(170, 17)
(361, 16)


In [98]:
df_total_cat.rename(columns={"variation_id_y": "variation_id"}, inplace=True)
df_total_cat.shape

(170, 17)

In [99]:
# condf(df_total_cat,'ml_inventory_id','WTHI04163')

In [100]:
df_total_cat = df_total_cat.drop_duplicates(subset=["ml_inventory_id"])

# df_total_cat[df_total_cat['ml_inventory_id']=='WTHI04163']

pergar ml_inventory_id e sum_total e acrescesntar essas informações no df de não catalogo pelo ml_inventory_id
ao fim deixar somatório do catalogo e não catalogo


In [101]:
# cols=['ml_inventory_id','total_sold_catalog']
# df_total_cat = df_total_cat[cols]
# df_total_cat.sample()

In [102]:
df_combined = pd.merge(
    df_total_not_cat,
    df_total_cat[["ml_inventory_id", "total_sold_catalog"]],
    on="ml_inventory_id",
    how="left",
)
# df_combined.sample()

In [103]:
# df_total_not_cat.shape

In [104]:
# condf(df_combined,'ml_inventory_id','UIUW89800')

In [105]:
# condf(df_combined,'ml_inventory_id','UIUW89800')

In [106]:
# df_combined['ml_inventory_id'].value_counts()

In [107]:
print(df_combined.shape)

(361, 17)


In [108]:
df = pd.merge(df_combined, df_stock_today, on="ml_inventory_id", how="left")
df["total_sold_catalog"] = df["total_sold_catalog"].fillna(0).astype("int64")

In [123]:
# df_combined

,ml_code,ml_inventory_id,value_name,variation_id,status,catalog_listing,logistic_type_x,seller_sku,order_id,title,shipping_id,variation_value_name,logistic_type_y,data,total_sold_quantity,total_sold_not_catalog,total_sold_catalog
0,MLB1435852259,TLCG38735,NaN,NaN,active,False,fulfillment,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,1.0
1,MLB1435836214,PUOF38682,NaN,NaN,active,False,fulfillment,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,1.0
2,MLB1400355425,ZXFT21237,NaN,NaN,active,False,fulfillment,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,NaN
3,MLB1440291959,JGRW38703,NaN,NaN,active,False,fulfillment,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,4.0
4,MLB1564998298,OVRP11252,NaN,NaN,paused,False,fulfillment,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
356,MLB1490210934,SPKV63227,nan,NaN,paused,False,fulfillment,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,NaN
357,MLB1556975175,DZVJ05686,Marrom,57895096313,active,False,fulfillment,FULLTAGIMA21KCOMCAPA,2000007119293852,Ukulelê Soprano Profissional Tagima 21k Mahoga...,42895838930,Marrom,fulfillment,2023-12-11,15,15,NaN
358,MLB950338822,MAEG54107,nan,NaN,paused,False,fulfillment,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,51.0
359,MLB1662970458,UIUW89800,Preto,64498487033,paused,False,fulfillment,FULLIZZO6480,2000007089127282,Pandeiro Original 10 Pol Abs Preto Com Pele Pr...,42882876401,Preto,fulfillment,2023-12-07,14,14,580.0


In [109]:
df.shape

(361, 26)

In [110]:
days = 30

df["total_sold"] = df["total_sold_catalog"] + df["total_sold_not_catalog"]
# qtd de produtos a enviar no período, caso seja valor negativo produto está acima do esperado para envio(sobrando)
df["period_send_fulfillment"] = np.ceil(
    (df["total_sold"] / df["days_available"]) * days - df["total_available_quantity"]
)

df["period_send_fulfillment"] = df["period_send_fulfillment"].fillna(0)

In [111]:
def calculate_percentual_send(row):
    if row["days_available"] != 0:
        if ((row["total_sold"] / row["days_available"]) * days) * 0.7 > row[
            "total_available_quantity"
        ]:
            return (row["total_sold"] / row["days_available"]) * days - row[
                "total_available_quantity"
            ]
    return 0


# Aplicando a função à coluna "percentual_send"
df["stock_replenishment"] = df.apply(calculate_percentual_send, axis=1)

In [112]:
df.sample(5)

,ml_code,ml_inventory_id,value_name,variation_id,status,catalog_listing,logistic_type_x,seller_sku,order_id,title,...,detail_quantity,references_id,references_variation_id,data_y,has_stock,days_available,total_available_quantity,total_sold,period_send_fulfillment,stock_replenishment
209,MLB949776093,QYGO52123,NaN,NaN,paused,False,fulfillment,NaN,NaN,NaN,...,NaN,NaN,NaN,NaT,NaN,NaN,NaN,0,0.0,0
275,MLB2636549148,MHIJ54971,NaN,NaN,active,False,fulfillment,NaN,NaN,NaN,...,NaN,NaN,NaN,NaT,NaN,NaN,NaN,19,0.0,0
219,MLB2006357721,RVHF10897,NaN,NaN,active,False,fulfillment,NaN,NaN,NaN,...,NaN,NaN,NaN,NaT,NaN,NaN,NaN,12,0.0,0
82,MLB1461737680,GLMK66383,NaN,NaN,active,False,fulfillment,NaN,NaN,NaN,...,NaN,NaN,NaN,NaT,NaN,NaN,NaN,3,0.0,0
127,MLB1435969309,TPDH38567,nan,NaN,paused,False,fulfillment,NaN,NaN,NaN,...,NaN,NaN,NaN,NaT,NaN,NaN,NaN,8,0.0,0


In [113]:
df.columns

Index(['ml_code', 'ml_inventory_id', 'value_name', 'variation_id', 'status',
       'catalog_listing', 'logistic_type_x', 'seller_sku', 'order_id', 'title',
       'shipping_id', 'variation_value_name', 'logistic_type_y', 'data_x',
       'total_sold_quantity', 'total_sold_not_catalog', 'total_sold_catalog',
       'available_quantity_today', 'detail_status', 'detail_quantity',
       'references_id', 'references_variation_id', 'data_y', 'has_stock',
       'days_available', 'total_available_quantity', 'total_sold',
       'period_send_fulfillment', 'stock_replenishment'],
      dtype='object')

In [114]:
df.shape

(361, 29)

In [121]:
# df

In [122]:
df_have_itens = df[df["days_available"] > 0]
df_have_itens.shape
df_have_itens

,ml_code,ml_inventory_id,value_name,variation_id,status,catalog_listing,logistic_type_x,seller_sku,order_id,title,...,detail_quantity,references_id,references_variation_id,data_y,has_stock,days_available,total_available_quantity,total_sold,period_send_fulfillment,stock_replenishment


In [116]:
df_sold_zero = df_have_itens[df_have_itens["total_sold"] == 0]
df_sold = df_have_itens[df_have_itens["total_sold"] > 0]

In [117]:
# produtos sem estoque no período
df_no_itens = df[df["days_available"] <= 0]
df_no_itens = df_no_itens.drop(columns=["period_send_fulfillment"])
df_no_itens.shape

(0, 28)

In [118]:
print(df_no_itens.shape)
print(df_sold.shape)
print(df_sold_zero.shape)

(0, 28)
(0, 29)
(0, 29)


In [119]:
df_sold_zero.sample(3)

ValueError: a must be greater than 0 unless no samples are taken

In [ ]:
df_sold.sample(3)

In [ ]:
dfx = df_have_itens.copy()

In [ ]:
df_sold_zero.columns

In [ ]:
dfx.columns

In [ ]:
cols = [
    "ml_code",
    "seller_sku",
    "ml_inventory_id",
    "value_name",
    "status",
    "title",
    "available_quantity_today",
    "detail_status",
    "detail_quantity",
    "total_available_quantity",
    "days_available",
    "total_sold_not_catalog",
    "total_sold_catalog",
    "total_sold",
    "period_send_fulfillment",
]

In [ ]:
col = [
    "ml_code",
    "seller_sku",
    "ml_inventory_id",
    "value_name",
    "status",
    "title",
    "total_sold_catalog",
    "available_quantity_today",
    "detail_status",
    "detail_quantity",
    "days_available",
    "total_available_quantity",
    "total_sold_not_catalog",
    "total_sold",
]

df_no_itens = df_no_itens[col]
df_no_itens

In [ ]:
def rename_columns(df):
    return df.rename(
        columns={
            "detail_status": "transfer_status",
            "detail_quantity": "transfer_quantity",
        }
    )


df_sold_zero = rename_columns(df_sold_zero)
df_sold = rename_columns(df_sold)
df_no_itens = rename_columns(df_no_itens)

In [ ]:
df_no_itens.columns

In [ ]:
dfx = dfx.rename(
    columns={"detail_status": "transfer_status", "detail_quantity": "transfer_quantity"}
)

In [ ]:
dfx.sample()
dfx.shape

In [ ]:
dfx

## Acabou!